Example PCIe endpoint re-enumrate if BAR is the same
```
echo 1 > /sys/bus/pci/devices/0000\:06\:00.0/remove
echo 1 > /sys/bus/pci/rescan
echo 1 > /sys/bus/pci/devices/0000\:06\:00.0/enable
```

In [1]:
from pcie_mem_util import *

In [2]:
pcie_offset = 0xC0000000 #not always the same
tlb_offset = 0xC0000000

In [3]:
bram_offset = 0xC0000000
bram_offset = bram_offset - tlb_offset + pcie_offset
mem = PcieMem(bram_offset)

In [4]:
hex(mem.wread(0))

'0xdeadbeef'

In [5]:
mem.wwrite(0,0xdeadbeef)

In [6]:
hex(mem.wread(0))

'0xdeadbeef'

In [7]:
mem.close()

Enable Power

In [71]:
en_offset = 0xC0010000
en_offset = en_offset - tlb_offset +pcie_offset
mem = PcieMem(en_offset)

In [72]:
mem.wwrite(0,0x0)

In [73]:
hex(mem.wread(0))

'0x0'

In [74]:
mem.wwrite(0,0x3)

In [75]:
hex(mem.wread(0))

'0x3'

In [76]:
mem.wwrite(0,0x7)

In [77]:
hex(mem.wread(0))

'0x7'

In [78]:
mem.close()

Turn on power

In [79]:
led_offset = 0xC0020000
led_offset = led_offset - tlb_offset +pcie_offset
mem = PcieMem(led_offset)

In [80]:
hex(mem.wread(0))

'0x1'

In [81]:
mem.wwrite(0,0x1)

In [82]:
hex(mem.wread(0))

'0x1'

In [83]:
mem.close()

I2Cs Acks

* Bit 0 -> SCL
* Bit 1 -> SDA

Addrs:
* ZL302601 -> 0x74
* MCP4432T-503E -> 0x2C
* MCP4728 -> 0x60

Read Tri-state at 0x4:
* 1 is input (open_drain)
* 0 is output (active drive)

Both inputs should be pulled high when inactive

Set SDA Low (Start)

Set SCL Low (Start)

Address bit6 set clk up and down

In [21]:
i2c_offset = 0xC0040000
i2c_offset = i2c_offset - tlb_offset + pcie_offset
i2c_mem = PcieMem(i2c_offset)

In [22]:
tap_offset = 0xC00C0000
tap_offset = tap_offset - tlb_offset + pcie_offset
tap_mem = PcieMem(tap_offset)

In [23]:
tap_mem.wwrite(0,0)

In [24]:
def sda (open):
    tap_mem.wwrite(0,1)
    #Read, Modify, write
    value = i2c_mem.wread(0x4)
    #keep only clk, and assign open
    value = (open<<1) | (value & 0x1)
    i2c_mem.wwrite(0x4,value)
    tap_mem.wwrite(0,0)

In [25]:
def scl (open):
    tap_mem.wwrite(0,1)
    #Read, Modify, write
    value = i2c_mem.wread(0x4)
    #keep only clk, and assign open
    value = (open<<0) | (value & 0x2)
    i2c_mem.wwrite(0x4,value)
    tap_mem.wwrite(0,0)

In [26]:
def toggle_scl ():
    scl(1)
    scl(0)

In [27]:
sda(0)
scl(0)
sda(1)
toggle_scl()
toggle_scl()
toggle_scl()
sda(0)
toggle_scl()
sda(1)
toggle_scl()
sda(0)
toggle_scl()
toggle_scl()
sda(0)
toggle_scl()
sda(1) # Also input
scl(1)
print("ACK:",hex(i2c_mem.wread(0x0)>>1))
scl(0)

scl(1)
sda(1)

ACK: 0x0


In [28]:
def i2c_start():
    sda(0)
    scl(0)

def i2c_stop():
    scl(1)
    sda(1)
    tap_mem.wwrite(0,1)    
    tap_mem.wwrite(0,0)
    tap_mem.wwrite(0,1)    
    tap_mem.wwrite(0,0)
    tap_mem.wwrite(0,1)    
    tap_mem.wwrite(0,0)
    tap_mem.wwrite(0,1)    
    tap_mem.wwrite(0,0)
    tap_mem.wwrite(0,1)    
    tap_mem.wwrite(0,0)

In [29]:
def i2c_wbyte(byte):
    for i in range(7,-1,-1):
        sda_bit = 0x1&(byte>>i)
        sda(sda_bit)
        toggle_scl()
    sda(1) # Also input
    scl(1)
    ack = (i2c_mem.wread(0x0)>>1)
    scl(0) 
    return ack


In [30]:
i2c_start()
print(i2c_wbyte(0x74<<1))
i2c_stop()

0


In [31]:
def i2c_rbyte(nack = 0):
    data = 0x0
    sda(1) # Also input
    for i in range(8):
        data = data << 1
        scl(1)
        data = data | (i2c_mem.wread(0x0)>>1)
        scl(0) 
    sda(nack)
    toggle_scl()
    return data

In [32]:
def i2c_mesg (addr, wbytes, rbytes=0):  #addr, byte [], number of reads
    i2c_start()
    ack = i2c_wbyte(addr<<1)
    for wbyte in wbytes:
        ack += i2c_wbyte(wbyte)    
    i2c_stop()
    if rbytes == 0:
        return ack    
    data = np.empty((0), dtype=np.uint8)
    i2c_start()
    ack = i2c_wbyte((addr<<1) | 0x1)
    for i in range(rbytes-1):
        data = np.append(data,i2c_rbyte())
    data = np.append(data,i2c_rbyte(nack=1))
    i2c_stop()
    return data
    
        
    

In [33]:
id = i2c_mesg(0x74,[0x03,0x00,0x30],2)

In [34]:
for value in id:
    print(hex(value))

0x1d
0x2


In [35]:
lock = i2c_mesg(0x74,[0x03,0x00,0x48],1)

In [36]:
for value in lock:
    print(hex(value))

0x0


In [37]:
wcmd = 0x02

In [38]:
pll_config = [[wcmd, 0x04, 0X23, 0x08],
                [wcmd, 0x00, 0X03, 0x01],
                [wcmd, 0x00, 0X04, 0x02],
                [wcmd, 0x00, 0X05, 0x21],
                [wcmd, 0x00, 0X07, 0x01],
                [wcmd, 0x01, 0X00, 0x42],
                [wcmd, 0x01, 0X01, 0x00],
                [wcmd, 0x01, 0X02, 0x01],
                [wcmd, 0x01, 0X06, 0x00],
                [wcmd, 0x01, 0X07, 0x00],
                [wcmd, 0x01, 0X08, 0x00],
                [wcmd, 0x01, 0X09, 0x00],
                [wcmd, 0x01, 0X0A, 0x20],
                [wcmd, 0x01, 0X0B, 0x03],
                [wcmd, 0x01, 0X21, 0x60],
                [wcmd, 0x01, 0X27, 0x90],
                [wcmd, 0x01, 0X41, 0x00],
                [wcmd, 0x01, 0X42, 0x00],
                [wcmd, 0x01, 0X43, 0x00],
                [wcmd, 0x01, 0X44, 0x00],
                [wcmd, 0x01, 0X45, 0xA0],
                [wcmd, 0x01, 0X53, 0x00],
                [wcmd, 0x01, 0X54, 0x50],
                [wcmd, 0x01, 0X55, 0xCE],
                [wcmd, 0x01, 0X80, 0x00],
                [wcmd, 0x02, 0X00, 0x80],
                [wcmd, 0x02, 0X01, 0x05],
                [wcmd, 0x02, 0X50, 0x80],
                [wcmd, 0x02, 0X51, 0x02],
                [wcmd, 0x04, 0X30, 0x0C],
                [wcmd, 0x04, 0X30, 0x00]]

In [39]:
for reg in pll_config:
    print(i2c_mesg(0x74,reg))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [40]:
pll_accquire = [[wcmd, 0x01, 0X00, 0x02],
                [wcmd, 0x01, 0X00, 0x42]]

In [41]:
for reg in pll_accquire:
    print(i2c_mesg(0x74,reg))

0
0


In [42]:
lock = i2c_mesg(0x74,[0x03,0x00,0x48],1)

In [43]:
for value in lock:
    print(hex(value))

0x8c


SPI Interface

In [44]:
spi_offset = 0xC0030000
spi_offset = spi_offset - tlb_offset + pcie_offset
spi_mem = PcieMem(spi_offset)

In [45]:
spi_mem.wwrite(0x4,0x0)

In [46]:
def spi_cs (index): #0 is clear
    tap_mem.wwrite(0,1)
    #Read, Modify, write
    value = spi_mem.wread(0x0)
    #keep only clk, and assign open
    if index > 0:
        value = (0x7c^(0x1<<(index+1))) | (value & 0x3)
    else:
        value = (0x7c) |(value & 0x3)
    spi_mem.wwrite(0x0,value)
    tap_mem.wwrite(0,0)

In [47]:
def spi_dat (dat): #0 is clear
    tap_mem.wwrite(0,1)
    #Read, Modify, write
    value = spi_mem.wread(0x0)
    #keep only clk, and assign open
    value = (dat<<1) | (value & 0b1111101)
    spi_mem.wwrite(0x0,value)
    tap_mem.wwrite(0,0)

In [48]:
def spi_clk (dat): #0 is clear
    tap_mem.wwrite(0,1)
    #Read, Modify, write
    value = spi_mem.wread(0x0)
    #keep only clk, and assign open
    value = (dat) | (value & 0b1111110)
    spi_mem.wwrite(0x0,value)
    tap_mem.wwrite(0,0)

In [49]:
def toggle_spi_clk ():
    spi_clk(1)
    spi_clk(0)

In [50]:
spi_cs(1)
spi_cs(0)

In [51]:
def spi_mesg (cs , reg, data):  # for adc
    spi_cs(cs)
    spi_clk(0)
    
    for i in range(7,-1,-1):        
        dat_bit = 0x1&(reg>>i)
        spi_dat(dat_bit)
        toggle_spi_clk()
        
    for i in range(15,-1,-1):         
        dat_bit = 0x1&(data>>i)
        spi_dat(dat_bit)
        toggle_spi_clk()
    
    spi_clk(1)
    spi_cs(0)
    spi_dat(1)

In [52]:
spi_mesg(1,0x0,0x1) # reset!

In [53]:
spi_mesg(1,0xf,0x200) # pwdn

In [54]:
spi_mesg(1,0x11,0x222) # lvds

In [55]:
spi_mesg(1,0x53,0x8) # lvds lclk

In [56]:
spi_mesg(1,0x42,0x60) # lvds ddr

In [57]:
spi_mesg(1,0x46,0x4) # lvds btc

In [58]:
spi_mesg(1,0x33,0x00) # lvds btc

In [59]:
spi_mesg(1,0xf,0x100) # pw sleep

Ramp Test

In [60]:
spi_mesg(1,0x25,0x40) # ramp-en

In [61]:
spi_mesg(1,0xf,0x40) # pw active

0's Test

In [62]:
spi_mesg(1,0x26,0x0000) # ramp-en

In [63]:
spi_mesg(1,0x27,0x0000) # ramp-en

In [64]:
spi_mesg(1,0x25,0x20) # ramp-en

Close all Mems

In [65]:
spi_mem.close()
i2c_mem.close()
tap_mem.close()